# Model evaluation

----------

Load all packages

In [1]:
#Import all packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Import all scikit-learn modules
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
#from sklearn.metrics import brier_score_loss
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#Set some global options
pd.set_option('max_columns',100) #Its nice to see all columns

#import warnings
#warnings.simplefilter("ignore", DeprecationWarning)

------------

The following are the steps to build and evaluate a model. Specifically we would like to optimize and compare models to find the best one.

## Step 1: Load data

We have our data in a pickle file which we load into a dataframe. 

In [2]:
df = pd.read_pickle('data/cleaned_df.pkl')
df['round'] = df['round'].astype(int)
df['year'] = df['year'].astype(int)

df.head()

,name,year,college,position,height,weight,fortyyd,vertical,bench,threecone,shuttle,broad,wonderlic,nflgrade,arms,hands,team,round,pick,overall,positiongroup
0,Ameer Abdullah,2015,Nebraska,RB,69.0,205.0,4.60,42.5,24.0,6.79,3.95,130.0,25.0,5.9,32.5,9.5,Lions,2,22.0,54.0,BA
1,Nelson Agholor,2015,Southern California,WR,72.0,198.0,4.42,33.0,12.0,7.19,4.35,114.0,25.0,5.6,32.5,9.5,Eagles,1,20.0,20.0,RE
2,Jay Ajayi,2015,Boise State,RB,72.0,221.0,4.57,39.0,19.0,7.10,4.10,121.0,25.0,6.0,32.5,9.5,Dolphins,5,13.0,149.0,BA
3,Kwon Alexander,2015,LSU,OLB,73.0,227.0,4.55,36.0,24.0,7.14,4.20,121.0,25.0,5.4,32.5,9.5,Buccaneers,4,25.0,124.0,LB
4,Mario Alford,2015,West Virginia,WR,68.0,180.0,4.43,34.0,13.0,6.64,4.07,121.0,25.0,5.3,32.5,9.5,Bengals,7,21.0,238.0,RE


Next, we drop unneeded columns and convert our one categorical column, positiongroup, into dummy columns. 

In [3]:
thedf = df.copy()
thedf['drafted'] = thedf['round'] > 0
thedf.drop(['name','college','position','pick','overall','team'], axis=1, inplace=True)
thedf.drop('round', axis=1, inplace=True)
thedf = pd.get_dummies(thedf, drop_first=True)
thedf

,year,height,weight,fortyyd,vertical,bench,threecone,shuttle,broad,wonderlic,nflgrade,arms,hands,drafted,positiongroup_DB,positiongroup_DL,positiongroup_LB,positiongroup_OL,positiongroup_QB,positiongroup_RE,positiongroup_ST
0,2015,69.0,205.0,4.60,42.5,24.0,6.79,3.95,130.0,25.0,5.9,32.5,9.5,True,0,0,0,0,0,0,0
1,2015,72.0,198.0,4.42,33.0,12.0,7.19,4.35,114.0,25.0,5.6,32.5,9.5,True,0,0,0,0,0,1,0
2,2015,72.0,221.0,4.57,39.0,19.0,7.10,4.10,121.0,25.0,6.0,32.5,9.5,True,0,0,0,0,0,0,0
3,2015,73.0,227.0,4.55,36.0,24.0,7.14,4.20,121.0,25.0,5.4,32.5,9.5,True,0,0,1,0,0,0,0
4,2015,68.0,180.0,4.43,34.0,13.0,6.64,4.07,121.0,25.0,5.3,32.5,9.5,True,0,0,0,0,0,1,0
5,2015,72.0,221.0,4.53,35.5,11.0,6.96,4.28,121.0,25.0,5.3,32.5,9.5,True,0,0,0,0,0,0,0
6,2015,72.0,218.0,4.56,35.5,21.0,7.09,4.03,122.0,25.0,5.5,32.5,9.5,True,1,0,0,0,0,0,0
7,2015,73.0,187.0,4.54,33.0,13.0,7.19,4.35,114.0,25.0,5.5,32.5,9.5,False,0,0,0,0,0,1,0
8,2015,78.0,294.0,4.97,30.0,21.0,7.20,4.19,111.0,25.0,5.6,32.5,9.5,True,0,1,0,0,0,0,0
9,2015,77.0,244.0,4.72,33.0,21.0,7.19,4.35,114.0,25.0,5.5,32.5,9.5,False,0,0,0,0,0,1,0


Finally, we make numpy arrays of our predictor variables and target variable. This x and y are our data. Note our target is a binary value, either a player is drafted or not.

In [4]:
target = 'drafted'
x = thedf.drop(target, axis=1).values
y = thedf[target].values

## Step 2: Split into train/test

We use train_test_split to separate our data into 80% training data and 20% testing data. We give a seed value so we keep consistency when running. We also stratify since our two classes are unbalanced, as can be seen below. 

In [5]:
#Percent of players drafted or not
thedf.groupby('drafted').size()*100./len(thedf)

drafted
False    36.321304
True     63.678696
dtype: float64

In [6]:
#Split into train/test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=1221, stratify=y)

In [7]:
print('The train data has %.0f rows which is %.2f%% of the total. ' % (len(x_train),len(x_train)*100./len(thedf)))
print('The test data has %.0f rows which is %.2f%% of the total. ' % (len(x_test),len(x_test)*100./len(thedf)))

The train data has 4810 rows which is 79.99% of the total. 
The test data has 1203 rows which is 20.01% of the total. 


## Step 3: Choose an estimator

Here, we will use three estimators or algorithms to model our data. First we will go through the simplest for this case, LogisticRegression, before going through the others.

In [8]:
logreg = LogisticRegression()

## Step 4: Choose a metric

There are a few metrics we could use. Ideally, we can test a few different metrics to see how they behave in our models. But we will choose a single metric here: the ROC AUC score.

In [9]:
score_func = make_scorer(roc_auc_score, greater_is_better=True)

## Step 5: Choose CV for estimator

Here I choose a stratified kfold scheme. This should help perform internal cross-validation on the training data we will feed it. There will be 5 folds. I enable shuffling to match test_train_split which also shuffles before splitting. I also use the stratified kfold method, and not the usual kfold, to also mirror the fact that we have unbalanced classes. We will also set a seed here, so we can consistently get the same folds. 

In [10]:
the_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1220)

## Step 6: Tune hyperparameters

Using GridSearchCV, we want to optimize the parameters to find the best LogisticRegression model for our data. We will feed the cv method with our decision from Step 4. A simple parameter grid is constructed to test both L1 and L2 penalties over various C values.

Note we will also use a pipeline to scale our data before we acutally fit it. The reason is that some of our columns have very different ranges and values. The year column has values from 1999 to 2015, while the fortyyd column has values from 4.2 to 6.1. Using a pipeline should make it easier to test out different scalers, if any. 

So let's first setup our grid.

In [11]:
param_grid = {'logreg__C': np.logspace(-3, 3, 10), 'logreg__penalty': ['l1', 'l2']}

Now we setup the pipeline, again with a scaler, for the logreg we already created. 

In [12]:
steps = [('scaler', StandardScaler()),
         ('logreg', logreg)]
pipeline = Pipeline(steps)

Then we make our GridSearchCV object with our created pipeline, parameter grid, cv method, and metric.

In [13]:
logreg_cv = GridSearchCV(pipeline, param_grid, cv=the_cv, scoring=score_func)

Now we can fit our train data with this GridSearchCV object. This will help us find the best parameters to get the best model we can.

In [14]:
logreg_cv.fit(x_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1220, shuffle=True),
       error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logreg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'logreg__C': array([  1.00000e-03,   4.64159e-03,   2.15443e-02,   1.00000e-01,
         4.64159e-01,   2.15443e+00,   1.00000e+01,   4.64159e+01,
         2.15443e+02,   1.00000e+03]), 'logreg__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=make_scorer(roc_auc_score), verbose=0)

Now we can print out that best model.

In [15]:
print('Best Logistic Reg Params: {}'.format(logreg_cv.best_params_))
print('Best Logistic Reg Score : %f' % logreg_cv.best_score_)

Best Logistic Reg Params: {'logreg__C': 2.154434690031882, 'logreg__penalty': 'l2'}
Best Logistic Reg Score : 0.630140


## Step 7: Get 'best' estimator

Here we use the best parameters from the GridSearchCV to create and fit the best LogisticRegression model.

In [16]:
#Create new 'best params' logreg
steps = [('scaler', StandardScaler() ),
         ('logreg', LogisticRegression(penalty=logreg_cv.best_params_['logreg__penalty'], 
                                       C=logreg_cv.best_params_['logreg__C'])
         )]
logreg_best = Pipeline(steps)

logreg_best.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('logreg', LogisticRegression(C=2.154434690031882, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

## Step 8: Evaluate the 'best' estimator

To evaluate the actual performance of the 'best' model we have, we must calculate the ROC AUC score. We can simply calculate the score over the test data (the stuff we held out in Step 2). This gives us a performance value on unseen data so it is independent of our previous calculation. 

In [17]:
roc_auc_score(y_test, logreg_best.predict_proba(x_test)[:,1])

0.69749239712972977

But, this should not be correct, right? We should really be using CV here as well to evaluate the performance and use cross_val_score. In fact we can use the exact same CV scheme too, namely 'the_cv,' within the cross_val_score function. This should give us a robust method to evaluate the score and evaluate the estimator.

In [18]:
cv_scores = cross_val_score(logreg_best, x_test, y_test, cv=the_cv, scoring='roc_auc')
print('CV Scores are {}'.format(cv_scores))
print('CV Score mean = %f and std = %f' % (cv_scores.mean(), cv_scores.std()))

CV Scores are [ 0.69871606  0.67520796  0.70265194  0.66846969  0.64811059]
CV Score mean = 0.678631 and std = 0.020134


Of the three metric measurements we have, all three are different. This is not unexpected since we are using different data for each measurement. However, I would have expected the differences to be much smaller, especially considering the gridsearch best score was so small compared to the cross validated score. 

In [19]:
scores_logreg = [logreg_cv.best_score_, roc_auc_score(y_test, logreg_best.predict_proba(x_test)[:,1]), 
                 cv_scores.mean()]
scores_logreg

[0.63013995336731032, 0.69749239712972977, 0.67863124652033002]

---------

Now we can repeat Steps 3 to 8 for each of our other two techniques. Practically speaking, Steps 4 and 5 remain the same: the metric and CV estimator will be re-used as-is. Let's do that now (with less text). 

### Support Vector Machine

In [20]:
#The estimator
svc = SVC(kernel='rbf', probability=True)

param_grid = {'svc__C': np.logspace(-2, 3, 3), 'svc__gamma' : [0.001, 0.01, 0.1]}
steps = [('scaler', StandardScaler()),
         ('svc', svc)]
pipeline = Pipeline(steps)

svc_cv = GridSearchCV(pipeline, param_grid, cv=the_cv, scoring=score_func)

In [21]:
svc_cv.fit(x_train, y_train)
print('Best SVC Params: {}'.format(svc_cv.best_params_))
print('Best SVC Score : %f' % svc_cv.best_score_)

Best SVC Params: {'svc__gamma': 0.001, 'svc__C': 1000.0}
Best SVC Score : 0.634047


In [22]:
#Create 'best' svc model
steps = [('scaler', StandardScaler()),
         ('svc', SVC(kernel='rbf', probability=True, C=svc_cv.best_params_['svc__C'], 
                     gamma=svc_cv.best_params_['svc__gamma']) 
         )]
svc_best = Pipeline(steps)

svc_best.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', SVC(C=1000.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [23]:
roc_auc_score(y_test, svc_best.predict_proba(x_test)[:,1])

0.7110311822239217

In [24]:
cv_scores = cross_val_score(svc_best, x_test, y_test, cv=the_cv, scoring='roc_auc')
print('CV Scores are {}'.format(cv_scores))
print('CV Score mean = %f and std = %f' % (cv_scores.mean(), cv_scores.std()))

CV Scores are [ 0.66300177  0.68508616  0.69363684  0.67478026  0.64210052]
CV Score mean = 0.671721 and std = 0.018009


In [25]:
scores_svc = [svc_cv.best_score_, roc_auc_score(y_test, svc_best.predict_proba(x_test)[:,1]), cv_scores.mean()]
scores_svc

[0.63404674572047814, 0.7110311822239217, 0.6717211087278393]

### Random Forest

In [26]:
#The estimator
rf = RandomForestClassifier(criterion='gini')

param_grid = {'rf__n_estimators' : [50, 100, 150], 'rf__min_samples_split' : [3, 4, 5], 
              'rf__max_features' : ['auto', None]}
steps = [('scaler', StandardScaler()),
         ('rf', rf)]
pipeline = Pipeline(steps)

rf_cv = GridSearchCV(pipeline, param_grid, cv=the_cv, scoring=score_func)

In [27]:
rf_cv.fit(x_train, y_train)
print('Best RF Params: {}'.format(rf_cv.best_params_))
print('Best RF Score : %f' % rf_cv.best_score_)

Best RF Params: {'rf__min_samples_split': 3, 'rf__n_estimators': 100, 'rf__max_features': None}
Best RF Score : 0.659083


In [28]:
#Create 'best' RF model
steps = [('scaler', StandardScaler()),
         ('rf', RandomForestClassifier(criterion='gini', n_estimators=rf_cv.best_params_['rf__n_estimators'],
                                       min_samples_split=rf_cv.best_params_['rf__min_samples_split'], 
                                       max_features=rf_cv.best_params_['rf__max_features'])
         )]
rf_best = Pipeline(steps)

rf_best.fit(x_train, y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=3, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])

In [29]:
roc_auc_score(y_test, rf_best.predict_proba(x_test)[:,1])

0.72015014548517964

In [30]:
cv_scores = cross_val_score(rf_best, x_test, y_test, cv=the_cv, scoring='roc_auc')
print('CV Scores are {}'.format(cv_scores))
print('CV Score mean = %f and std = %f' % (cv_scores.mean(), cv_scores.std()))

CV Scores are [ 0.69200118  0.71301248  0.71223049  0.73007287  0.6663286 ]
CV Score mean = 0.702729 and std = 0.021835


In [31]:
scores_rf = [rf_cv.best_score_, roc_auc_score(y_test, rf_best.predict_proba(x_test)[:,1]), cv_scores.mean()]
scores_rf

[0.65908296409376677, 0.72015014548517964, 0.70272912337728866]

--------------

## Step 9: Comparison

So we can make a comparison between the 'best' models of each technique. Recall, the last number in the array is the CV-score. By this evaluation, Random Forest seems best. 

In [32]:
print('LR  : {}'.format(scores_logreg))
print('SVC : {}'.format(scores_svc))
print('RF  : {}'.format(scores_rf))

LR  : [0.63013995336731032, 0.69749239712972977, 0.67863124652033002]
SVC : [0.63404674572047814, 0.7110311822239217, 0.6717211087278393]
RF  : [0.65908296409376677, 0.72015014548517964, 0.70272912337728866]


## Step 10: Train final model

Now that we have finalized our model, we train it over the whole dataset. This is our final model which we can use for future predictions. 